# Import Modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os
import numpy as np
import torch
from torch.autograd import Variable
from src.get_nets import PNet, RNet, ONet
from src.box_utils import nms, calibrate_box, get_image_boxes, convert_to_square
from src.first_stage import run_first_stage
import torch.nn as nn

# Path Definitions

In [5]:
dataset_path = '../Dataset/emotiw/'

processed_dataset_path = '../Dataset/FaceFeatures/'

# MTCNN Model Definition for Extracting Face Features

In [3]:
pnet = PNet()
rnet = RNet()
onet = ONet()
onet.eval()

ONet(
  (features): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=32)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=64)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=64)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv4): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
    (prelu4): PReLU(num_parameters=128)
    (flatten): Flatten()
    (conv5): Linear(in_features=1152, out_features=256, bias=True)
    (drop5): Dropout(p=0.25)
    (prelu5): PReLU(num_parameters=256)
  )
  (conv6_1): Linear(in_features=256, out_features=2, bias=True)
  (conv6_2): Linear(in_features=256, out_features=4, bias=True)
  (co

In [4]:
class OnetFeatures(nn.Module):
    def __init__(self, original_model):
        super(OnetFeatures, self).__init__()
        self.features = nn.Sequential(*list(onet.children())[:-3])
        
    def forward(self, x):
        x = self.features(x)
        return x

def get_face_features(image, min_face_size=20.0,
                 thresholds=[0.6, 0.7, 0.8],
                 nms_thresholds=[0.7, 0.7, 0.7]):
    """
    Arguments:
        image: an instance of PIL.Image.
        min_face_size: a float number.
        thresholds: a list of length 3.
        nms_thresholds: a list of length 3.

    Returns:
        two float numpy arrays of shapes [n_boxes, 4] and [n_boxes, 10],
        bounding boxes and facial landmarks.
    """

    # LOAD MODELS
    pnet = PNet()
    rnet = RNet()
    onet = ONet()
    onet.eval()

    # BUILD AN IMAGE PYRAMID
    width, height = image.size
    min_length = min(height, width)

    min_detection_size = 12
    factor = 0.707  # sqrt(0.5)

    # scales for scaling the image
    scales = []

    # scales the image so that
    # minimum size that we can detect equals to
    # minimum face size that we want to detect
    m = min_detection_size/min_face_size
    min_length *= m

    factor_count = 0
    while min_length > min_detection_size:
        scales.append(m*factor**factor_count)
        min_length *= factor
        factor_count += 1

    # STAGE 1

    # it will be returned
    bounding_boxes = []

    # run P-Net on different scales
    for s in scales:
        boxes = run_first_stage(image, pnet, scale=s, threshold=thresholds[0])
        bounding_boxes.append(boxes)

    # collect boxes (and offsets, and scores) from different scales
    bounding_boxes = [i for i in bounding_boxes if i is not None]
    bounding_boxes = np.vstack(bounding_boxes)

    keep = nms(bounding_boxes[:, 0:5], nms_thresholds[0])
    bounding_boxes = bounding_boxes[keep]

    # use offsets predicted by pnet to transform bounding boxes
    bounding_boxes = calibrate_box(bounding_boxes[:, 0:5], bounding_boxes[:, 5:])
    # shape [n_boxes, 5]

    bounding_boxes = convert_to_square(bounding_boxes)
    bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])

    # STAGE 2

    img_boxes = get_image_boxes(bounding_boxes, image, size=24)
    img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
    output = rnet(img_boxes)
    offsets = output[0].data.numpy()  # shape [n_boxes, 4]
    probs = output[1].data.numpy()  # shape [n_boxes, 2]

    keep = np.where(probs[:, 1] > thresholds[1])[0]
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
    offsets = offsets[keep]

    keep = nms(bounding_boxes, nms_thresholds[1])
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes = calibrate_box(bounding_boxes, offsets[keep])
    bounding_boxes = convert_to_square(bounding_boxes)
    bounding_boxes[:, 0:4] = np.round(bounding_boxes[:, 0:4])

    # STAGE 3

    img_boxes = get_image_boxes(bounding_boxes, image, size=48)
    if len(img_boxes) == 0: 
        return [], []
    img_boxes = Variable(torch.FloatTensor(img_boxes), volatile=True)
    output = onet(img_boxes)
    
    faceFeatureModel = OnetFeatures(onet)

    featureOutputs = faceFeatureModel(img_boxes)
    
    landmarks = output[0].data.numpy()  # shape [n_boxes, 10]
    offsets = output[1].data.numpy()  # shape [n_boxes, 4]
    probs = output[2].data.numpy()  # shape [n_boxes, 2]

    keep = np.where(probs[:, 1] > thresholds[2])[0]
    bounding_boxes = bounding_boxes[keep]
    bounding_boxes[:, 4] = probs[keep, 1].reshape((-1,))
    offsets = offsets[keep]
    landmarks = landmarks[keep]

    bounding_boxes = calibrate_box(bounding_boxes, offsets)
    keep = nms(bounding_boxes, nms_thresholds[2], mode='min')
    
    featureOutputs = featureOutputs[keep]

    return featureOutputs

# Load Train and Val Dataset

In [6]:
image_datasets = {x : datasets.ImageFolder(os.path.join(dataset_path, x))
                    for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class_names

['Negative', 'Neutral', 'Positive']

In [8]:
training_dataset = image_datasets['train']
validation_dataset = image_datasets['val']

In [9]:
neg_train = sorted(os.listdir(dataset_path + 'train/Negative/'))
neu_train = sorted(os.listdir(dataset_path + 'train/Neutral/'))
pos_train = sorted(os.listdir(dataset_path + 'train/Positive/'))

neg_val = sorted(os.listdir(dataset_path + 'val/Negative/'))
neu_val = sorted(os.listdir(dataset_path + 'val/Neutral/'))
pos_val = sorted(os.listdir(dataset_path + 'val/Positive/'))

In [10]:
neg_train_filelist = [x.split('.')[0] for x in neg_train]
neu_train_filelist = [x.split('.')[0] for x in neu_train]
pos_train_filelist = [x.split('.')[0] for x in pos_train]

neg_val_filelist = [x.split('.')[0] for x in neg_val]
neu_val_filelist = [x.split('.')[0] for x in neu_val]
pos_val_filelist = [x.split('.')[0] for x in pos_val]


In [11]:
neg_train_filelist = neg_train_filelist[1:]

neg_val_filelist = neg_val_filelist[1:]
neu_val_filelist = neu_val_filelist[1:]
pos_val_filelist = pos_val_filelist[1:]

In [30]:
print(neg_train_filelist[:10])
print(neu_train_filelist[:10])
print(pos_train_filelist[:10])

print(neg_val_filelist[:10])
print(neu_val_filelist[:10])
print(pos_val_filelist[:10])

['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']
['neg_1', 'neg_10', 'neg_100', 'neg_1000', 'neg_1001', 'neg_1002', 'neg_1003', 'neg_1004', 'neg_1005', 'neg_1006']
['neu_1', 'neu_10', 'neu_100', 'neu_1000', 'neu_1001', 'neu_1002', 'neu_1003', 'neu_1004', 'neu_1005', 'neu_1006']
['pos_1', 'pos_10', 'pos_100', 'pos_1000', 'pos_1001', 'pos_1002', 'pos_1003', 'pos_1004', 'pos_1005', 'pos_1006']


In [31]:
train_filelist = neg_train_filelist + neu_train_filelist + pos_train_filelist
val_filelist = neg_val_filelist + neu_val_filelist + pos_val_filelist

In [32]:
print(len(training_dataset))
print(len(validation_dataset))


9815
4346


# Extract Face Features

In [34]:
for i in range(len(validation_dataset)):
    image, label = validation_dataset[i]
    print(val_filelist[i])
    try:
        if label == 0:
            if os.path.isfile(processed_dataset_path + 'val/Negative/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()

            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Negative/' + val_filelist[i] , a=features)
            
        elif label == 1:
            if os.path.isfile(processed_dataset_path + 'val/Neutral/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()
            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Neutral/' + val_filelist[i] , a=features)
            
        else:
            if os.path.isfile(processed_dataset_path + 'val/Positive/' + val_filelist[i] + '.npz'):
                print(val_filelist[i] + ' Already present')
                continue
            features = get_face_features(image)
            if(type(features)) == tuple:
                with open('hello.text', 'a') as f:
                    f.write(val_filelist[i])
                continue
            features = features.data.numpy()
            if features.size == 0:
                print('MTCNN model handling empty face condition at ' + val_filelist[i])
            np.savez(processed_dataset_path + 'val/Positive/' + val_filelist[i] , a=features)            
            
    except ValueError:
        print('No faces detected for ' + val_filelist[i] + ". Also MTCNN failed.")
        if label == 0:
            np.savez(processed_dataset_path + 'val/Negative/' + val_filelist[i] , a=np.zeros(1))
        elif label == 1:
            np.savez(processed_dataset_path + 'val/Neutral/' + val_filelist[i] , a=np.zeros(1))
        else:
            np.savez(processed_dataset_path + 'val/Positive/' + val_filelist[i] , a=np.zeros(1))
        continue

neg_1
neg_10
neg_100
MTCNN model handling empty face condition at neg_100
neg_1000
neg_1001
neg_1002
neg_1003
neg_1004
neg_1005
neg_1006
neg_1007
neg_1008
neg_1009
MTCNN model handling empty face condition at neg_1009
neg_101
neg_1010
neg_1011
neg_1012
neg_1013
neg_1014
MTCNN model handling empty face condition at neg_1014
neg_1015
neg_1016
neg_1017
neg_1018
neg_1019
neg_102
neg_1020
neg_1021
neg_1022
neg_1023
neg_1024
neg_1025
neg_1026
neg_1027
neg_1028
neg_1029
neg_103
neg_1030
neg_1031
neg_1032
neg_1033
neg_1034
neg_1035
neg_1036
neg_1037
neg_1038
neg_1039
neg_104
neg_1040
neg_1041
neg_1042
neg_1043
neg_1044
neg_1045
neg_1046
neg_1047
neg_1048
neg_1049
neg_105
neg_1050
neg_1051
neg_1052
neg_1053
neg_1054
neg_1055
neg_1056
neg_1057
neg_1058
neg_1059
neg_106
neg_1060
neg_1061
neg_1062
neg_1063
neg_1064
neg_1065
neg_1066
neg_1067
neg_1068
neg_1069
neg_107
neg_1070
neg_1071
neg_1072
neg_1073
neg_1074
neg_1075
neg_1076
neg_1077
neg_1078
neg_1079
neg_108
neg_1080
neg_1081
neg_1082
neg_108

neg_754
neg_755
neg_756
neg_757
neg_758
MTCNN model handling empty face condition at neg_758
neg_759
neg_76
neg_760
neg_761
neg_762
neg_763
neg_764
neg_765
neg_766
neg_767
neg_768
neg_769
neg_77
neg_770
neg_771
neg_772
neg_773
neg_774
neg_775
neg_776
MTCNN model handling empty face condition at neg_776
neg_777
neg_778
neg_779
neg_78
neg_780
neg_781
neg_782
neg_783
neg_784
neg_785
neg_786
neg_787
neg_788
neg_789
neg_79
neg_790
neg_791
neg_792
neg_793
neg_794
neg_795
neg_796
neg_797
neg_798
neg_799
neg_8
neg_80
neg_800
neg_801
neg_802
neg_803
neg_804
neg_805
neg_806
neg_807
neg_808
neg_809
neg_81
neg_810
neg_811
neg_812
neg_813
neg_814
neg_815
neg_816
neg_817
neg_818
neg_819
neg_82
neg_820
neg_821
neg_822
neg_823
neg_824
neg_825
neg_826
neg_827
neg_828
neg_829
neg_83
neg_830
neg_831
neg_832
neg_833
neg_834
neg_835
neg_836
neg_837
neg_838
neg_839
neg_84
neg_840
neg_841
neg_842
MTCNN model handling empty face condition at neg_842
neg_843
neg_844
neg_845
neg_846
neg_847
neg_848
neg_849
neg_

neu_307
neu_308
neu_309
neu_31
neu_310
neu_311
neu_312
neu_313
neu_314
neu_315
neu_316
neu_317
neu_318
neu_319
neu_32
neu_320
neu_321
neu_322
neu_323
neu_324
neu_325
neu_326
neu_327
neu_328
neu_329
neu_33
neu_330
neu_331
neu_332
neu_333
neu_334
neu_335
neu_336
neu_337
neu_338
neu_339
neu_34
neu_340
neu_341
neu_342
neu_343
neu_344
neu_345
neu_346
neu_347
neu_348
neu_349
neu_35
neu_350
neu_351
neu_352
neu_353
neu_354
neu_355
neu_356
neu_357
neu_358
neu_359
MTCNN model handling empty face condition at neu_359
neu_36
neu_360
neu_361
neu_362
neu_363
neu_364
neu_365
neu_366
neu_367
neu_368
neu_369
neu_37
neu_370
neu_371
neu_372
neu_373
neu_374
neu_375
neu_376
neu_377
neu_378
neu_379
neu_38
neu_380
neu_381
neu_382
neu_383
neu_384
neu_385
neu_386
neu_387
neu_388
neu_389
neu_39
neu_390
neu_391
neu_392
neu_393
neu_394
neu_395
neu_396
neu_397
neu_398
MTCNN model handling empty face condition at neu_398
neu_399
neu_4
neu_40
neu_400
neu_401
neu_402
neu_403
neu_404
neu_405
neu_406
neu_407
neu_408
ne

pos_1183
pos_1184
pos_1185
pos_1186
pos_1187
pos_1188
pos_1189
pos_119
pos_1190
pos_1191
pos_1192
pos_1193
pos_1194
pos_1195
pos_1196
pos_1197
pos_1198
pos_1199
pos_12
pos_120
pos_1200
pos_1201
pos_1202
pos_1203
pos_1204
pos_1205
pos_1206
pos_1207
pos_1208
pos_1209
pos_121
pos_1210
pos_1211
pos_1212
pos_1213
pos_1214
pos_1215
pos_1216
pos_1217
pos_1218
pos_1219
pos_122
pos_1220
pos_1221
pos_1222
pos_1223
pos_1224
pos_1225
pos_1226
pos_1227
pos_1228
pos_1229
pos_123
pos_1230
pos_1231
pos_1232
pos_1233
pos_1234
pos_1235
pos_1236
pos_1237
pos_1238
pos_1239
pos_124
pos_1240
pos_1241
pos_1242
pos_1243
pos_1244
pos_1245
pos_1246
pos_1247
pos_1248
pos_1249
pos_125
pos_1250
pos_1251
pos_1252
pos_1253
pos_1254
pos_1255
pos_1256
pos_1257
pos_1258
pos_1259
pos_126
pos_1260
pos_1261
pos_1262
pos_1263
pos_1264
pos_1265
pos_1266
pos_1267
pos_1268
pos_1269
pos_127
pos_1270
pos_1271
pos_1272
pos_1273
pos_1274
pos_1275
pos_1276
pos_1277
pos_1278
pos_1279
pos_128
pos_1280
pos_1281
pos_1282
pos_1283
pos_

pos_438
pos_439
pos_44
pos_440
pos_441
pos_442
pos_443
pos_444
pos_445
pos_446
pos_447
pos_448
pos_449
pos_45
pos_450
pos_451
pos_452
pos_453
pos_454
pos_455
pos_456
pos_457
pos_458
pos_459
pos_46
pos_460
pos_461
pos_462
pos_463
pos_464
pos_465
pos_466
pos_467
pos_468
pos_469
pos_47
pos_470
pos_471
pos_472
pos_473
pos_474
pos_475
pos_476
pos_477
pos_478
pos_479
pos_48
pos_480
pos_481
pos_482
pos_483
pos_484
pos_485
pos_486
pos_487
pos_488
pos_489
pos_49
pos_490
pos_491
pos_492
pos_493
pos_494
pos_495
pos_496
pos_497
pos_498
pos_499
pos_5
pos_50
pos_500
pos_501
pos_502
pos_503
pos_504
pos_505
pos_506
pos_507
pos_508
pos_509
pos_51
pos_510
pos_511
pos_512
pos_513
pos_514
pos_515
pos_516
pos_517
pos_518
pos_519
pos_52
pos_520
pos_521
pos_522
pos_523
pos_524
pos_525
pos_526
pos_527
pos_528
pos_529
pos_53
pos_530
pos_531
pos_532
pos_533
pos_534
pos_535
pos_536
pos_537
pos_538
pos_539
pos_54
pos_540
pos_541
pos_542
pos_543
pos_544
pos_545
pos_546
pos_547
pos_548
pos_549
pos_55
pos_550
pos_55